# Bokeh - plot a lightcurve

In this notebook, I share a slightly simplified example from an article about a flare on a giant star that I'm currently working on.
My goal is not to reproduce the bokeh documentation (click on any of the example at http://bokeh.org/) but to show one real-world example with real-world astronomy data.

In [1]:
from astropy.table import QTable, Table
from astropy.time import Time
from astropy import timeseries

import numpy as np

In [2]:
# ASAS lightcurve is downloaded from archive and then
# updated by triggering processing of most recent observations on
# from https://asas-sn.osu.edu/?dec=12.76423&ra=91.06242
asassn = QTable.read('../data/ASAS-SN.csv', format='ascii')
# Filter out bad data
asassn = asassn[asassn['mag'] < 90]
asassn['time'] = Time(asassn['HJD'], format='jd')
asassng = timeseries.TimeSeries(asassn[asassn['Filter'] == 'g'])

In [3]:
# Some datapoints have large uncertainties, e.g. due to thinn clouds in the sky.
# Let's filter them out.
asassng = asassng[asassng['mag_err'] < 0.007]

In [4]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Span, Label

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, Whisker
from bokeh.plotting import figure, show, output_notebook, output_file


Bokeh's native output is on the web and normally it will open a new tab in the webbrowser for every new plot. However, since we are working in the Jupyter notebook, which is also a web application, we can also embed the plots directly into the notebook. This is done by calling the `output_notebook()` function.

In [5]:
output_notebook()

Loading BokehJS ...

Bokeh uses a specific data structure for its plots called `ColumnDataSource`. You can think of it as a dictionary that contains all the data that you want to plot, though in reality it is a bit more complicated than that. Details are in the bokeh documentation, but for most uses it's enough to know that you can create a `ColumnDataSource` from a dictionary.

In most places, you could also just pass in numpy arrays or pandas dataframes - I'll have an example of that later.

In [6]:
source = ColumnDataSource(
    {'date': asassng.time.datetime64,   # These are are datatime objects, nut just numbers
     'ASAS-SN': asassng['mag'],  # This is just a numpy array of numbers
     })

OK, let's make the first plot with mostly default settings. We do need to tell the figure that the x-axis comes in datatime objects, not in simple numbers.

In [7]:

fig = figure(x_axis_type="datetime")


fig.circle('date', 'ASAS-SN', # keys in the ColumnDataSource for the columns used to plot x and y
           source=source,  # the data source we're using
           legend_label='ASAS-SN g', 
           color='black',  # Some setting for how to plot. 
                           # Of course, we can also change the size of the circles, etc.
           )
fig.yaxis.axis_label = 'mag'
fig.xaxis.axis_label = 'Observation date'
fig.y_range.flipped = True  # Magnitudes are a strange unit, where smaller numbers mean brighter objects.
                            # So, we flip the y-axis to make it more intuitive.

show(fig)

This is already an interactive plot. Using your mouse and the toolbar on the right you can zoom in and out, pan around, and save the plot. Note how the display of the dates on the x-axis changes when you zoom in and out.

Bokeh has lots of options to customize how the plot looks and the documentation has plenty of examples - look there for how to do it.

## More data in other bands
To make it more interesting, let's add more data from other observatories.

In [8]:
aavso = Table.read('../data/AAVSO-ASASJ060415+12459.txt', format='ascii')
aavso['time'] = Time(aavso['JD'], format='jd')

The most common bands in this dataset are B, N, I, R, so let's just use those for the plot.

In [9]:
aavso = timeseries.TimeSeries(aavso[np.isin(aavso['Band'], ['V', 'B', 'R', 'I'])])

In [10]:
fig = figure(height=300, width=800,
           x_axis_type="datetime",
           background_fill_color="#efefef",  # Just to make it look a little different, give the plot a grey background
           x_range=(asassng.time.datetime64[10], asassng.time.datetime64[-1]))



fig.circle('date', 'ASAS-SN', source=source, legend_label='ASAS-SN g', color='black')
# Observations are done with different filters (called "bands" in astronomy).
# Let's plot them in different colors.
for b in 'BVRI':
    ind = aavso['Band'] == b
        
    fig.circle(
        # Instead of using a ColumnDataSource, we can also just pass numpy arrays directly
        aavso['time'][ind].datetime64, aavso['Magnitude'][ind], 
                   color={'V': 'green', 'B': 'blue', 'R': 'orange', 'I': 'red'}[b],
                   legend_label=f"AAVSO {b}")
    
fig.yaxis.axis_label = 'mag'
fig.y_range.flipped = True
fig.legend.location = "top_left"

show(fig)

This plot looks good and we can zoom and pan around as in the last one.

Can we do better? Probably. There really is no great need to zoom in y, but on the other hand, when you zoom in x, it's hard to see what's going on over the whole range. It would be nice to keep a small version of the full ightcurve an highlight the regions we've zoomed in on. 

Fortunately, bokeh has a tool for just that, so we don't have to write a lot of code. We simply have to make two panels, plot our data into both of them and then link the x-axis together with the `RangeTool`.

Before we do the plot though, we'll tell bokeh that we don't just want to see the plot in the notebook - we also want a copy of it as a standalone html file. This is done by calling `output_file()`.

In [11]:
# For development, I just want a display that works without worrying about the file path
mode = 'cdn'

# For sending to a journal, I need a file path and then include the js files into the package that we send.
# So generate figures with relative file path and then edit those manually to match what I send.
#mode = 'relative'
output_file('lc_opt.html', title='Lightcurve', mode=mode)

In [12]:

p = figure(height=300, width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(asassng.time.datetime64[1900],
                                                     asassng.time.datetime64[2100]))


select = figure(title="Drag the middle and edges of the selection box to change the range above",
                height=130, width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

for fig in [p, select]:
    fig.circle('date', 'ASAS-SN', source=source, legend_label='ASAS-SN g', color='black')
    # Observations are done with different filters (called "bands" in astronomy).
    # Let's plot them in different colors.
    for b in 'BVRI':
        ind = aavso['Band'] == b
        
        fig.circle(
            # Instead of using a ColumnDataSource, we can also just pass numpy arrays directly
            aavso['time'][ind].datetime64, aavso['Magnitude'][ind], 
                   color={'V': 'green', 'B': 'blue', 'R': 'orange', 'I': 'red'}[b],
                   legend_label=f"AAVSO {b}")
    fig.yaxis.axis_label = 'mag'
    fig.y_range.flipped = True

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2
select.add_tools(range_tool)
select.legend.visible = False

show(column(p, select))